In [50]:
import os, sklearn, math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from joblib import dump, load

tf.random.set_seed(42)

data = '211102_Data'
dataname = data+'_0206_1'
filename = 'myData/'+data+'.csv'
SD = pd.read_csv(filename)
SD=SD.iloc[:,:28]
SensorData = SD

In [51]:
SDext =SD # SensorData[SensorData['z']==0]
mask = SDext["z"].isin(list(range(37,127,3)))
SD = SDext[mask]

In [52]:
dic = {'Sx00':'Lx10','Sx01':'Lx11','Sx02':'Lx12','Sx03':'Lx13','Sx08':'Lx14','Sx09':'Lx15','Sx0A':'Lx16','Sx0B':'Lx17',
       'Sx10':'Lx18','Sx11':'Lx19','Sx12':'Lx1A','Sx13':'Lx1B',
       'Sx07':'Lx00','Sx06':'Lx01','Sx05':'Lx02','Sx04':'Lx03','Sx0F':'Lx04','Sx0E':'Lx05','Sx0D':'Lx06','Sx0C':'Lx07',
      'Sx17':'Lx08','Sx16':'Lx09','Sx15':'Lx0A','Sx14':'Lx0B'} # 센서 데이터를 로케이션에 매칭시키는 딕셔너리
SD = SD.rename(columns = dic)
SD = SD[['Lx00','Lx01','Lx02','Lx03','Lx04','Lx05','Lx06','Lx07','Lx08','Lx09','Lx0A','Lx0B',
         'Lx10','Lx11','Lx12','Lx13','Lx14','Lx15','Lx16','Lx17','Lx18','Lx19','Lx1A','Lx1B','r','theta','theta_enc','z']] # 순서대로 정렬

In [53]:
import numpy as np
toRad = np.pi/180 
SD['cos']=np.cos(SD.theta_enc*toRad)
SD['sin']=np.sin(SD.theta_enc*toRad)
SD['theta_enc'] = SD['theta_enc'] - 180

In [54]:
k = SD.iloc[:,[26,27,28,29]]
k = k.to_numpy()

In [55]:
SD

,Lx00,Lx01,Lx02,Lx03,Lx04,Lx05,Lx06,Lx07,Lx08,Lx09,...,Lx18,Lx19,Lx1A,Lx1B,r,theta,theta_enc,z,cos,sin
103683,55.46,38.75,56.95,62.85,56.50,30.26,36.77,49.01,70.31,47.34,...,25.96,54.84,63.57,47.6,0.9,0,-49.70,37,-0.64679,0.762668
103684,53.66,32.10,57.82,61.91,56.50,30.26,37.77,49.01,70.31,49.38,...,25.96,53.10,63.57,47.6,0.9,0,-49.70,37,-0.64679,0.762668
103685,53.66,37.80,58.69,59.09,53.83,31.94,36.77,51.86,70.31,50.40,...,30.51,51.36,63.57,47.6,0.9,0,-49.70,37,-0.64679,0.762668
103686,55.46,35.90,56.08,60.03,55.61,31.10,35.77,50.91,69.42,49.38,...,29.60,51.36,63.57,47.6,0.9,0,-49.70,37,-0.64679,0.762668
103687,51.86,37.80,55.21,61.91,55.61,30.26,36.77,50.91,68.53,49.38,...,29.60,53.97,65.39,48.6,0.9,0,-49.70,37,-0.64679,0.762668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449280,57.26,41.60,56.95,47.81,56.50,31.94,36.77,51.86,70.31,52.44,...,28.69,52.23,60.84,47.6,1.8,355,8.44,124,-0.98917,-0.146774
449281,58.16,36.85,57.82,48.75,55.61,28.58,31.77,51.86,70.31,53.46,...,28.69,52.23,62.66,47.6,1.8,355,8.44,124,-0.98917,-0.146774
449282,57.26,40.65,57.82,50.63,56.50,30.26,35.77,50.91,70.31,52.44,...,27.78,50.49,62.66,48.6,1.8,355,8.44,124,-0.98917,-0.146774
449283,56.36,40.65,58.69,50.63,57.39,31.94,35.77,53.76,67.64,51.42,...,28.69,53.10,61.75,46.6,1.8,355,8.44,124,-0.98917,-0.146774


In [56]:
SD = (SD- SD.mean())/SD.std() # 모든 데이터 정규화

In [57]:
SD.describe()

,Lx00,Lx01,Lx02,Lx03,Lx04,Lx05,Lx06,Lx07,Lx08,Lx09,...,Lx18,Lx19,Lx1A,Lx1B,r,theta,theta_enc,z,cos,sin
count,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,...,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05,3.456020e+05
mean,-3.198438e-11,-2.517560e-13,-2.368865e-13,2.278445e-12,-1.654787e-11,3.353380e-11,1.789559e-10,-3.036679e-11,-5.278324e-11,-4.684535e-11,...,8.079606e-12,2.967846e-12,-2.412812e-11,8.862090e-11,-5.878738e-12,-5.335400e-16,-5.485032e-15,2.766290e-17,1.813590e-15,-2.604178e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-8.251533e+00,-5.171854e+00,-8.652200e+00,-8.147050e+00,-8.846871e+00,-3.929217e+00,-4.601332e+00,-8.281495e+00,-8.450994e+00,-8.695327e+00,...,-7.353136e+00,-8.208968e+00,-8.103383e+00,-7.925454e+00,-1.341634e+00,-1.708137e+00,-1.720055e+00,-1.675249e+00,-1.407485e+00,-1.428794e+00
25%,-7.501968e-01,-6.091842e-01,-5.219459e-01,-3.540284e-01,-6.470161e-01,-8.302276e-01,-8.764825e-01,-3.237509e-01,-3.335143e-01,-3.070024e-01,...,-6.157818e-01,-4.126107e-01,-2.678628e-01,-4.773031e-01,-1.118027e+00,-8.901514e-01,-8.679470e-01,-8.665082e-01,-1.001282e+00,-9.979589e-01
50%,8.328499e-02,-1.746442e-01,2.910795e-01,1.998672e-04,1.729694e-01,5.519786e-02,-2.556742e-01,1.183459e-01,5.303242e-02,5.770739e-02,...,1.328131e-01,-4.135560e-02,1.239132e-01,1.998015e-01,-4.472078e-01,-2.404853e-02,-9.581329e-03,0.000000e+00,3.071208e-03,1.962849e-02
75%,5.000259e-01,6.944358e-01,6.975922e-01,7.086564e-01,5.829622e-01,9.406233e-01,3.651341e-01,5.604428e-01,4.395791e-01,4.224172e-01,...,5.071106e-01,7.011546e-01,5.156892e-01,5.383538e-01,4.472181e-01,8.420544e-01,8.637066e-01,8.665082e-01,1.002746e+00,9.939545e-01
max,3.833953e+00,4.822566e+00,2.730156e+00,3.188254e+00,3.452912e+00,4.482325e+00,4.710792e+00,3.655121e+00,3.145406e+00,2.975386e+00,...,3.875788e+00,3.671195e+00,2.866345e+00,2.908220e+00,1.341644e+00,1.708157e+00,1.744119e+00,1.675249e+00,1.413627e+00,1.407174e+00


In [58]:
rawSD = SD.iloc[:,:24] # 센서데이터만 자름
#rawSD['Lx00_end'] = rawSD['Lx00']
#rawSD['Lx01_end'] = rawSD['Lx01']
y = SD.iloc[:,[27,28,29]] # z cos sin 짜름

In [59]:
arrSD= rawSD.to_numpy() # raw data를 넘파이 차원으로 바꿈
import numpy as np
arr2dSD = np.reshape(arrSD,(-1,2,12,1)) # 2x12 배열로 바꿈
X = arr2dSD

In [60]:
X.shape

(345602, 2, 12, 1)

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.25, random_state=42)

In [77]:
import cv2
n= 100

a = max(map(max,arr2dSD[n]))
b = min(map(min,arr2dSD[n]))

test = -arr2dSD[n]*256/(a[0]-b[0])

img_source = test

height, width = img_source.shape[:2]
img_result = cv2.resize(img_source, (40*width, 40*height), interpolation = cv2.INTER_NEAREST )

noimage = np.zeros(img_result.shape[:2],img_result.dtype)
avg = cv2.mean(img_result)[0]/2.0

img_result = cv2.addWeighted(img_result, 1.0, noimage, 0, -avg)

print(k[n])

cv2.imshow("img",img_result)
cv2.imwrite('myCNN_fig/img_result_{0}.jpg'.format(a),img_result)
cv2.waitKey(0)

[-49.7         37.          -0.64678978   0.76266833]


-1

In [63]:
k[12311]

array([-20.04      ,  40.        ,  -0.93945362,   0.34267609])

In [75]:
a = max(map(max,arr2dSD[100]))
b = min(map(min,arr2dSD[100]))
a[0]-b[0]

4.0560120927506995